In [2]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd
from pyswip import Prolog
 
# Διάβασμα του αρχείου 'movie_metadata.csv' 
path = r"C:\Users\OWNER\Desktop\AI Σειρα 2\2ο Θέμα"
data = pd.read_csv(path + "\movies_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,...,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,Enter the World of Pandora.,Avatar,7.2,11800,2009.0,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007.0,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,A Plan No One Escapes,Spectre,6.3,4466,2015.0,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,English,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,The Legend Ends,The Dark Knight Rises,7.6,9106,2012.0,United States of America,Christopher Nolan,Michael Caine,Gary Oldman,Anne Hathaway
4,4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,English,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"Lost in our world, found in another.",John Carter,6.1,2124,2012.0,United States of America,Andrew Stanton,Lynn Collins,Samantha Morton,Willem Dafoe


In [75]:
def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

In [4]:
import pandas as pd
from pyswip import Prolog

def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

#create World
#Ορίζουμε τον κόσμο μας
prolog = Prolog()

#Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
 
for row in data.itertuples(index=True, name='Pandas'):
  movie_title = clean_text(getattr(row, 'movie_title'))

#ids epeidh oi titloi Out of the blue, Batman, The host yparxoun parapanw apo 2 fores
  ident = getattr(row, 'id')
  literals.append("identification('"+ movie_title +"','"+ str(ident) +"')")

#plot keywords
  for plotkeys in getattr(row, 'plot_keywords').split("|"):
    plot = clean_text(plotkeys)
    literals.append("plot('"+ str(ident) +"','"+ plot +"')")

#language
  lang = getattr(row, 'language')
  literals.append("language('"+ str(ident) +"','"+ lang +"')")

#genre
  for genre in getattr(row, 'genres').split("|"):
    literals.append("genre('"+ str(ident) +"','"+ genre +"')")

#popularity
  pop = getattr(row, 'popularity')
  literals.append("popularity('"+ str(ident) +"','"+ str(pop) +"')")  # den evala int alla string

#production company
  comp = getattr(row, 'production_companies')
  if comp != '[]':
    arr = comp.replace('[','').replace('}]','').split('}, ')
    for s in arr:
      s = clean_text(s).replace('{name: ','').replace(', id: ',',').split(',')[0]
      literals.append("production_company('"+ str(ident) +"','"+ s +"')")
  else:
    literals.append("production_company('"+ str(ident) +"','""')") #indie

#production country
  countries = getattr(row, 'production_countries')
  if countries != '[]':
    countries = countries.replace('[','').replace('}]','').split('}, ')
    for s in countries:
      s = clean_text(s).replace('name: ','').replace(', id: ',',').split(',')[1]
      literals.append("production_country('"+ str(ident) +"','"+ s +"')")
  else:
    literals.append("production_country('"+ str(ident) +"','""')")

#release date
  date = getattr(row, 'release_date')
  literals.append("release_date('"+ str(ident) +"','"+ date +"')")

#duration
  dur = getattr(row, 'duration')
  literals.append("duration('"+ str(ident) +"','"+ str(dur) +"')")

#languages spoken
  slang = getattr(row, 'spoken_languages')
  if slang != '[]':
    slang = slang.replace('[','').replace('}]','').split('}, ')
    for s in slang:
      s = clean_text(s).replace('name: ','').replace(', id: ',',').split(',')[1]
      literals.append("spoken_languages('"+ str(ident) +"','"+ s +"')")
  else:
    literals.append("spoken_languages('"+ str(ident) +"','""')")

#reviews
  avg = getattr(row, 'vote_average')
  literals.append("vote_average('"+ str(ident) +"','"+ str(avg) +"')")

#year
  year = getattr(row, 'title_year')
  literals.append("title_year('"+ str(ident) +"','"+ str(year) +"')")

#director
  director = getattr(row, 'director_name')
  literals.append("director('"+ str(ident) +"','"+ clean_text(director) +"')")

#actor1
  actor1 = getattr(row, 'actor_1_name')
  literals.append("actor_1('"+ str(ident) +"','"+ clean_text(actor1) +"')")

#actor2
  actor2 = getattr(row, 'actor_2_name')
  literals.append("actor_2('"+ str(ident) +"','"+ clean_text(actor2) +"')")

#actor3
  actor3 = getattr(row, 'actor_3_name')
  literals.append("actor_3('"+ str(ident) +"','"+ clean_text(actor3) +"')")
    
#Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
  prolog.assertz(literal)
  print (literal +'.')


PrologError: Caused by: 'assertz((actor_1('100','Dexter Fletcher'))).'. Returned: 'error(permission_error(modify, static_procedure, /(actor_1, 2)), context(:(system, /(assertz, 1)), _2484))'.

In [1]:
import pandas as pd
from pyswip import Prolog

prolog = Prolog()
changeme = "/Users/OWNER/Desktop/testdrive.pl"
prolog.consult(changeme)

In [2]:
q = prolog.query("vs_genre('Avatar',L)")
a = next(q)
a = {variable: [str(elem) for elem in a[variable]] for variable in a}
q.close()
print(a['L'])

['2012', '47 Ronin', '9', 'A Sound of Thunder', 'AVP: Alien vs. Predator', 'After Earth', 'Aliens in the Attic', 'Aliens vs Predator: Requiem', 'Anacondas: The Hunt for the Blood Orchid', 'Ant-Man', 'Armageddon', 'Avengers: Age of Ultron', 'Babylon A.D.', 'Batman v Superman: Dawn of Justice', 'Battleship', 'Beastmaster 2: Through the Portal of Time', 'Big Trouble in Little China', 'Bill & Teds Bogus Journey', 'Blade: Trinity', 'BloodRayne', 'Brave', 'Captain America: Civil War', 'Captain America: The First Avenger', 'Captain America: The Winter Soldier', 'Cirque du Freak: The Vampires Assistant', 'Clash of the Titans', 'Conan the Barbarian', 'Conan the Destroyer', 'Congo', 'Damnation Alley', 'Daybreakers', 'Deep Rising', 'Delgo', 'Divergent', 'Dragon Wars: D-War', 'Dragonball Evolution', 'Dune', 'Dungeons & Dragons: Wrath of the Dragon God', 'E.T. the Extra-Terrestrial', 'Enders Game', 'Eragon', 'Escape from L.A.', 'Fantastic 4: Rise of the Silver Surfer', 'Fantastic Four', 'Final Fant

In [95]:
def ps_recommender(movie):
    q1 = prolog.query("vs_genre('" + movie +"',L)")
    vsg = next(q1)
    vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}
    q1.close()
    
    q2 = prolog.query("s_char('" + movie +"',L)")
    sa = next(q2)
    sa = {variable: [str(elem) for elem in sa[variable]] for variable in sa}    
    q2.close()
    
    q3 = prolog.query("es_plot('" + movie +"',L)")
    esp = next(q3)
    esp = {variable: [str(elem) for elem in esp[variable]] for variable in esp}    
    q3.close()
    
    L1 = vsg['L']
    L2 = sa['L']
    L3 = esp['L']
    
    vsg_sa = []
    recom = []
    
    for movie in L1:
        if movie in L2:
            vsg_sa.append(movie)
    for movie in vsg_sa:
        if movie in L3:
            recom.append(movie)
            L3.remove(movie)
            vsg_sa.remove(movie)
            
    answer = recom + vsg_sa + L3
     
    return answer

In [96]:
ps_recommender('Harry Potter and the Half-Blood Prince')

['Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Philosophers Stone',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Philosophers Stone']

In [97]:
L = [1,2,3]
L.append(4)
L.remove(2)
L

[1, 3, 4]

In [124]:
def es_recommender(movie):
    q1 = prolog.query("vs_genre('" + movie +"',L)")
    vsg = next(q1)
    vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}
    q1.close()
    
    q2 = prolog.query("s_char('" + movie +"',L)")
    sa = next(q2)
    sa = {variable: [str(elem) for elem in sa[variable]] for variable in sa}    
    q2.close()
    
    q3 = prolog.query("vs_plot('" + movie +"',L)")
    vsp = next(q3)
    vsp = {variable: [str(elem) for elem in vsp[variable]] for variable in vsp}    
    q3.close()
    
    L1 = vsg['L']
    L2 = sa['L']
    L3 = vsp['L']
    
    vsg_sa = []
    recom = []
    
    for movie in L1:
        if movie not in L2 and movie in L3:
            vsg_sa.append(movie)
            L3.remove(movie)

            
    answer = vsg_sa + L3
     
    return answer

In [125]:
es_recommender('Harry Potter and the Half-Blood Prince')

['Oz: The Great and Powerful',
 'Spirited Away',
 'Conan the Destroyer',
 'Goosebumps',
 'Hotel Transylvania 2',
 'Hotel Transylvania',
 'In the Name of the King: A Dungeon Siege Tale',
 'Practical Magic',
 'The Craft',
 'The Huntsman: Winters War',
 'The Sorcerers Apprentice',
 'Warlock']

In [126]:
def vs_recommender(movie):
    q1 = prolog.query("vs_genre('" + movie +"',L)")
    vsg = next(q1)
    vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}
    q1.close()

    q2 = prolog.query("vs_plot('" + movie +"',L)")
    vsp = next(q2)
    vsp = {variable: [str(elem) for elem in vsp[variable]] for variable in vsp}    
    q2.close()
    
    q3 = prolog.query("s_plot('" + movie +"',L)")
    sp = next(q3)
    sp = {variable: [str(elem) for elem in sp[variable]] for variable in sp}    
    q3.close()
    
    q4 = prolog.query("es_plot('" + movie +"',L)")
    esp = next(q4)
    esp = {variable: [str(elem) for elem in esp[variable]] for variable in esp}    
    q4.close()
    
    q5 = prolog.query("s_genre('" + movie +"',L)")
    sg = next(q5)
    sg = {variable: [str(elem) for elem in sg[variable]] for variable in sg}
    q5.close()
    
    L1 = vsg['L']
    L2 = vsp['L']
    L3 = sp['L']
    L4 = esp['L']
    L5 = sg['L']
    
    helper = []
    recom = []
    
    for movie in L1:
        if movie in L3:
            helper.append(movie)
            L3.remove(movie)
            L1.remove(movie)
    for movei in L1:
        if movei in L2 or movei in L4:
            L1.remove(movei)
            
    for movie in helper:
        if movie in L5:
            helper.remove(movie)
            
    answer = helper + L1
     
    return answer

In [127]:
vs_recommender('Harry Potter and the Half-Blood Prince')

['A Simple Wish',
 'Asterix at the Olympic Games',
 'Last Action Hero',
 'Pan',
 'Shrek 2',
 'Shrek',
 'Stardust',
 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe',
 'The NeverEnding Story',
 'The Smurfs',
 'The Thief and the Cobbler',
 'The Wizard of Oz',
 'Time Bandits',
 'A Monster in Paris',
 'Aladdin',
 'Alice in Wonderland',
 'Aliens in the Attic',
 'Arthur and the Invisibles',
 'Babe: Pig in the City',
 'Bill & Teds Bogus Journey',
 'Brave',
 'Charlie and the Chocolate Factory',
 'City of Ember',
 'Delgo',
 'Dragon Nest: Warriors Dawn',
 'E.T. the Extra-Terrestrial',
 'Epic',
 'Eragon',
 'G-Force',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Philosophers Stone',
 'Herbie Fully Loaded',
 'Hook',
 'How to Train Your Dragon 2',
 'How to Train Your Dragon',
 'Inkheart',
 'Jimmy Neutron: Boy Genius',
 'Jonah: A VeggieTales Movie',
 'Legend of the Guardians: The Owls of GaHoole',
 'Maleficent',
 'Mighty Joe Young',
 'Mirror Mirror',
 'Night at the M

In [128]:
def s_recommender(movie):
    q1 = prolog.query("s_genre('" + movie +"',L)")
    sg = next(q1)
    sg = {variable: [str(elem) for elem in sg[variable]] for variable in sg}
    q1.close()

    q2 = prolog.query("vs_plot('" + movie +"',L)")
    vsp = next(q2)
    vsp = {variable: [str(elem) for elem in vsp[variable]] for variable in vsp}    
    q2.close()
    
    q3 = prolog.query("s_plot('" + movie +"',L)")
    sp = next(q3)
    sp = {variable: [str(elem) for elem in sp[variable]] for variable in sp}    
    q3.close()
    
    q4 = prolog.query("es_plot('" + movie +"',L)")
    esp = next(q4)
    esp = {variable: [str(elem) for elem in esp[variable]] for variable in esp} 
    q4.close()
    
    q5 = prolog.query("vs_genre('" + movie +"',L)")
    vsg = next(q5)
    vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}
    q5.close()    
    
    L1 = sg['L']
    L2 = vsp['L']
    L3 = sp['L']
    L4 = esp['L']
    L5 = vsg['L']
    
    helper = []
    recom = []
    
    for movie in L3:
        if movie in L5:
            L3.remove(movie)
            
    for movie in L1:
        if movie in L2  or movie in L4:
            L1.remove(movie)
    
    for movie in L1:
        if movie in L3:
            L1.remove(movie)
            L3.remove(movie)
            helper.append(movie)   
            
    answer = helper + L3 + L1
     
    return answer

In [129]:
s_recommender('Harry Potter and the Half-Blood Prince')

['Big Fish',
 'Big Trouble in Little China',
 'Bogus',
 'Cinderella',
 'Ella Enchanted',
 'Happily NEver After',
 'Hocus Pocus',
 'Howls Moving Castle',
 'Mary Poppins',
 'Nanny McPhee',
 'Pirates of the Caribbean: Dead Mans Chest',
 'Seventh Son',
 'Snow White and the Huntsman',
 'Teen Wolf Too',
 'The 13th Warrior',
 'The Covenant',
 'The Haunted Mansion',
 'The Last Witch Hunter',
 'The Mortal Instruments: City of Bones',
 'The Santa Clause 2',
 'The Twilight Saga: Eclipse',
 'Underworld: Rise of the Lycans',
 'Warcraft',
 'A Very Long Engagement',
 'An American Haunting',
 'Army of Darkness',
 'Beautiful Creatures',
 'Being John Malkovich',
 'Blood and Chocolate',
 'Book of Shadows: Blair Witch 2',
 'Charlie Bartlett',
 'Crazy/Beautiful',
 'Cursed',
 'Dark Shadows',
 'Darkness Falls',
 'Deceptive Practice: The Mysteries and Mentors of Ricky Jay',
 'DragonHeart',
 'Dragonslayer',
 'Dune',
 'Dylan Dog: Dead of Night',
 'Four Rooms',
 'Fur: An Imaginary Portrait of Diane Arbus',
 'Han

In [130]:
L1 = vs_recommender('Harry Potter and the Half-Blood Prince')
L2 = s_recommender('Harry Potter and the Half-Blood Prince')
for i in L1:
    if i in L2:
        print(i)

Shrek
The Thief and the Cobbler
Time Bandits
The Chronicles of Narnia: The Voyage of the Dawn Treader


In [131]:
movie = 'Harry Potter and the Half-Blood Prince'
q1 = prolog.query("s_genre('" + movie +"',L)")
sg = next(q1)
sg = {variable: [str(elem) for elem in sg[variable]] for variable in sg}
q1.close()

q2 = prolog.query("vs_genre('" + movie +"',L)")
vsg = next(q2)
vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}
q2.close()

L = 'Shrek'
L in vsg['L']

True

In [132]:
L = [1,2,3,4]
L.remove(1)
L

[2, 3, 4]

In [133]:
def venn(movie):
    q1 = prolog.query("s_genre('" + movie +"',L)")
    sg = next(q1)
    sg = {variable: [str(elem) for elem in sg[variable]] for variable in sg}
    q1.close()
    
    q2 = prolog.query("vs_genre('" + movie +"',L)")
    vsg = next(q2)
    vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}
    q2.close()
    
    q3 = prolog.query("es_plot('" + movie +"',L)")
    esp = next(q3)
    esp = {variable: [str(elem) for elem in esp[variable]] for variable in esp} 
    q3.close()
    
    q4 = prolog.query("vs_plot('" + movie +"',L)")
    vsp = next(q4)
    vsp = {variable: [str(elem) for elem in vsp[variable]] for variable in vsp} 
    q4.close()
    
    q5 = prolog.query("s_plot('" + movie +"',L)")
    sp = next(q5)
    sp = {variable: [str(elem) for elem in sp[variable]] for variable in sp} 
    q5.close()
    
    SG = sg['L']
    VSG = vsg['L']
    ESP = esp['L']
    VSP = vsp['L']
    SP = sp['L']
    
    esp_sg = []
    vsp_sg = []
    sp_sg = []
    
    esp_vsg = []
    vsp_vsg = []
    sp_vsg = []
    
    for movie in ESP:
        if movie in SG:
            ESP.remove(movie)
            SG.remove(movie)
            esp_sg.append(movie)
        if movie in VSG:
            ESP.remove(movie)
            VSG.remove(movie)
            esp_vsg.append(movie)
    for movie in VSP:
        if movie in SG:
            VSP.remove(movie)
            SG.remove(movie)
            vsp_sg.append(movie)
        if movie in VSG:
            VSP.remove(movie)
            VSG.remove(movie)
            vsp_vsg.append(movie)
    for movie in SP:
        if movie in SG:
            SP.remove(movie)
            SG.remove(movie)
            sp_sg.append(movie)
        if movie in VSG:
            SP.remove(movie)
            VSG.remove(movie)
            sp_vsg.append(movie)
            
    return SG,VSG,ESP,VSP,SP, esp_sg, vsp_sg, sp_sg,esp_vsg, vsp_vsg, sp_vsg 

In [134]:
A,B,C,D,E,F,G,H,I,J,K = venn('Harry Potter and the Half-Blood Prince')
for movie in A:
    if movie in B or movie in C or movie in D or movie in E or movie in F or movie in G or movie in H or movie in I or movie in J or movie in K:
        print(movie)

Big Trouble in Little China
Hocus Pocus
In the Name of the King: A Dungeon Siege Tale
Mary Poppins
Snow White and the Seven Dwarfs
The 13th Warrior
The Twilight Saga: New Moon
Warcraft
Warlock


In [135]:
L = [1,2,3,4]
L.remove(2)
print(L)

[1, 3, 4]


In [136]:
movie = {}
movie['abc'] = 1
movie

{'abc': 1}

In [137]:
score_weights = {i:i + 1 for i in range(4)}
score_weights

{0: 1, 1: 2, 2: 3, 3: 4}

In [146]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import random

rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
score_weights = {i:i + 1 for i in range(4)} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender

def train_recommender(ratings, rating_weights, score_weights, number_of_movies = 10):
    """
    Στην συνάρτηση αυτή μπορούμε να ορίζουμε ποιο υποσύνολο των αξιολογήσεων θα χρησιμοποιήσουμε για το train μαζί με τα βάρη ομοιότητας και σκορ
    Σε συνδυασμό με τον αριθμό των ταινιών που θέλουμε να χρησιμοποιήσουμε σαν σύνολο δεδομένων π.χ. 10 από τις 100 ή 3 από τις 100 κ.ο.κ
    Αν θέλουμε να χρησιμοποιήσουμε όλες τις ταινίες σαν training set τότε ορίζουμε το number_of_movies = - 1
    """

    if number_of_movies > len(ratings):
        number_of_movies = len(ratings)


    if number_of_movies != -1:
        indexes = random.sample(range(len(ratings)), number_of_movies)
        ratings = ratings.iloc[indexes]

    movie_score = {}
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        ps_movies = ps_recommender(movie)
        es_movies = es_recommender(movie)
        vs_movies = vs_recommender(movie)
        s_movies = vs_recommender(movie)
        
        for similar_movie in es_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[3]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[3]
                
        for similar_movie in ps_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[2]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[2]
                
        for similar_movie in vs_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[1]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[1]
                
        for similar_movie in vs_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[0]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[0]
        
    return movie_score


#αυτό είναι ένα παράδειγμα για το πως θα μπορούσε να υλοποιήθει η predict
#έχουμε ορίσει ότι μια ταινία θα έπρεπε να είναι προτεινόμενη αν είχε σκορ > 0
def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: #αν έχουμε σχηματίσει βαθμολογία για την ταινία αυτή
            pred.append(int(movie_score[movie] > 0)) #heuristic για το αν μια ταινία είναι προτεινόμενη
            real.append(int(rating > 3))# έτσι ορίζουμε ότι μια ταινία θα έπρεπε να είναι προτεινόμενη
            #η συνθήκη αυτή δεν μπορεί να αλλάξει
        else: #δεν μπορούμε να προτείνουμε κάτι για το όποιο δεν έχουμε σχηματίσει εικόνα
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics

In [143]:
path = 'C:\\Users\\OWNER\\Desktop\\AI Σειρα 2\\2ο Θέμα'

train_ratings = pd.read_csv(path + "\\train_ratings.csv")
test_ratings = pd.read_csv(path + "\\test_ratings.csv")

In [147]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

precision: 0.40417021197193603
recall: 0.09861111111111112
f1: 0.15286293994879246


In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")